In [1]:
import numpy as np
import time
import Robot
from MyRobotMath import SE3

In [2]:
scara = Robot.SCARA(2,1,1)
print(scara.S_tw)
print(type(scara.S_tw))

M = scara.zero
S = scara.S_tw
B = scara.B_tw

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, -1.0, 0.0], [0.0, 0.0, 1.0, 0.0, -2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
<class 'list'>


In [3]:
se3 = SE3()
w = se3.skew(scara.B_tw[0][:3])
v = scara.B_tw[0][3:]
print(w)
print(v)

[[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  0]]
[0, 2, 0]


In [4]:
es_1 = se3.matexp('R',45,S[0])
es_2 = se3.matexp('R',45,S[1])
es_3 = se3.matexp('R',45,S[2])
es_4 = se3.matexp('P',0.9,S[3])
print(es_1)
matexps_s = [es_1,es_2,es_3,es_4]


[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [5]:
eb_1 = se3.matexp('R',45,B[0])
eb_2 = se3.matexp('R',45,B[1])
eb_3 = se3.matexp('R',45,B[2])
eb_4 = se3.matexp('P',0.9,B[3])
print(eb_1)
matexps_b = [eb_1,eb_2,eb_3,eb_4]


[[ 0.70710678 -0.70710678  0.         -0.58578644]
 [ 0.70710678  0.70710678  0.          1.41421356]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [6]:
js = se3.space_jacobian(matexps_s,scara.S_tw)
print(js)

[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 1.          1.          1.          0.        ]
 [ 0.          0.70710678  1.70710678  0.        ]
 [ 0.         -0.70710678 -0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]


In [7]:
jb = se3.body_jacobian(M,matexps_b,matexps_s,scara.S_tw)
print(jb)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 1.70710678e+00  7.07106781e-01  3.31400081e-17  0.00000000e+00]
 [ 7.07106781e-01  7.07106781e-01 -1.67155782e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
print(se3.matsb(M,matexps_b))

[[-0.70710678 -0.70710678  0.          0.70710678]
 [ 0.70710678 -0.70710678  0.          1.70710678]
 [ 0.          0.          1.          0.9       ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
def select_init(desired, preset):
    x, y = desired[0], desired[1]
    key = (x > 0, y > 0)
    mapping = {
        (True,   True): 0,
        (False,  True): 1,
        (False, False): 2,
        (True,  False): 3,
    }
    return preset[mapping[key]]

desired = [0.6,0.4,0,0,0,142] # 목표 자세의 (x,y,z,roll,pitch,yaw)
T_d = se3.pose_to_SE3(desired) # 목표 자세의 Tranformation Matrix
threshold = 1e-3 # 오차 범위
count = 0

init = select_init(desired, scara.preset) # 목표 자세에 해당하는 사분면 초기 상태 호출

# while True:

#     count += 1 # 연산 횟수 증가

eb_1 = se3.matexp('R',init[0],B[0])
eb_2 = se3.matexp('R',init[1],B[1])
eb_3 = se3.matexp('R',init[2],B[2])
eb_4 = se3.matexp('P',init[3],B[3])

matexps_b = [eb_1,eb_2,eb_3,eb_4] # Body Axis 기준 각 축의 Matrix Exponential

es_1 = se3.matexp('R',init[0],S[0])
es_2 = se3.matexp('R',init[1],S[1])
es_3 = se3.matexp('R',init[2],S[2])
es_4 = se3.matexp('P',init[3],S[3])

matexps_s = [es_1,es_2,es_3,es_4] # Space Axis 기준 각 축의 Matrix Exponential

T_sb = se3.matsb(M,matexps_b) # Forward Kinematics 적용 변환행렬
print(T_sb)
currentPosition = []
for i in range(3):
    currentPosition.append(T_sb[i,3].item()) # 현재 x, y, z

eulerAngles = se3.CurrenntAxis(T_sb)
# print(eulerAngles)
for eulerAngle in eulerAngles:
    currentPosition.append(eulerAngle) # 현재 Euler 각도
# print(currentPosition)

relativePosition = np.array(desired) - np.array(currentPosition) # 목표와 추정값의 오차
# print(relativePosition)

T_bd = np.dot(np.linalg.inv(T_sb),T_d) # Relative Trasformation Matrix

J_b = se3.body_jacobian(M,matexps_b,matexps_s,S) # Body Jacobian
# print(J_b)
J_pseudo = se3.j_inv(J_b) # Jacobian의 역행렬 (또는 의사역행렬)
# print(J_pseudo)
V_bd = se3.relativetwist(T_bd) # Ralative Twist

theta = np.zeros(4)
theta[:3], theta[3] = np.array(np.deg2rad(init[:3])), init[3]
thetak = theta.reshape(4,1) + J_pseudo @ V_bd.reshape(6,1) # Newton Raphson Method
thetak[:3] = np.rad2deg(thetak[:3])

init = thetak.flatten().tolist()
for i in range(3):
    if init[i] % 360 > 180:
        init[i] = init[i] % 360 - 360
    else:
        init[i] = init[i] % 360
# print(sum(init[:3]))
# print(init)

# if np.all(np.abs(relativePosition) < threshold): # 오차가 임계값 이내면 break
#     print(currentPosition)
#     print(f"연산 횟수 : {count}, 관절 각도 : {init}")
#     break

# time.sleep(0.001)

[[ 0.79863551 -0.60181502  0.          1.59727102]
 [ 0.60181502  0.79863551  0.          1.20363005]
 [ 0.          0.          1.          0.8       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.91027682e-17 -3.82055364e-17  0.00000000e+00]
 [ 2.00000000e+00  1.00000000e+00 -2.33468665e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.18672342e-17  0.00000000e+00 -1.66666667e-01  1.59189735e-17
   5.00000000e-01  0.00000000e+00]
 [-7.50549794e-18  0.00000000e+00  3.33333333e-01 -1.27351788e-17
   1.53527128e-16  0.00000000e+00]
 [-2.68782301e-17  0.00000000e+00  8.33333333e-01 -4.13893311e-17
  -5.00000000e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00

In [10]:
a = [-543.325462869557, -180.07172692049105, 865.3971897900482, 0.0]
print(a[0]%360)

176.674537130443


In [11]:
print(np.random.normal(0,2))

3.2005175392878855
